# Data wrangling and datetimes

Dates and times are a special kind of data type. In this notebook, we will keep exploring the `orders` and `orderlines` datasets from Eniac and learn to deal with them.

In [20]:
import pandas as pd
df = pd.read_csv('data/eniac/orderlines.csv')
df.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38


## Data exploration

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


Two variables need to be modified: 

* `unit_price`: it is detected as an object but it has to be a float. Why is that happening? 
* `date`: needs to be transformed tp a date time format. 

For the moment we will only focus on the `date`. Since it contains both the date and the time, we will transform the data type using the pandas method `to_datetime`: 

In [85]:
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.7+ MB


In [86]:
df.describe()

,id,id_order,product_id,product_quantity
count,2.939830e+05,293983.000000,293983.0,293983.000000
mean,1.397918e+06,419999.116544,0.0,1.121126
std,1.530096e+05,66344.486479,0.0,3.396569
min,1.119109e+06,241319.000000,0.0,1.000000
25%,1.262542e+06,362258.500000,0.0,1.000000
50%,1.406940e+06,425956.000000,0.0,1.000000
75%,1.531322e+06,478657.000000,0.0,1.000000
max,1.650203e+06,527401.000000,0.0,999.000000


You can count the quantity of observations with `.value_counts()`.

In [87]:
df['sku'].value_counts()

MIC0036      6282
APP1216      5627
APP0662      5445
APP1190      5039
APP0663      3942
             ... 
APP2110         1
TWS0053-A       1
APP1837         1
UAG0026         1
REP0299         1
Name: sku, Length: 7951, dtype: int64

Exercise: check if the dates from the `orderlines` dataset match with the dates on the `orders` dataset.

In [2]:
orders_df = pd.read_csv('data/eniac/orders.csv')

In [103]:
merged = df.merge(orders_df,how='inner',left_on='id_order',right_on='order_id')
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293749 entries, 0 to 293748
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293749 non-null  int64         
 1   id_order          293749 non-null  int64         
 2   product_id        293749 non-null  int64         
 3   product_quantity  293749 non-null  int64         
 4   sku               293749 non-null  object        
 5   unit_price        293749 non-null  object        
 6   date              293749 non-null  datetime64[ns]
 7   order_id          293749 non-null  int64         
 8   created_date      293749 non-null  object        
 9   total_paid        293743 non-null  float64       
 10  state             293749 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(5), object(4)
memory usage: 26.9+ MB


In [90]:
merged['date'].equals(merged['created_date'])

False

## Aggregating with pandas

* Grouping and aggregating is one of the main ways to explore data. The main tools to do that with pandas are:
    * [`pandas.DataFrame.groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html).
    * [`pandas.DataFrame.agg()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html?highlight=agg#pandas.DataFrame.agg) 
        * [`pandas.DataFrame.count()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html)

In [91]:
df.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38


How many id do we have on each sku?

In [92]:
df.groupby(['sku']).agg({'product_quantity':'count'})

,product_quantity
sku,
8MO0001-A,2
8MO0003-A,3
8MO0007,29
8MO0008,30
8MO0009,28
...,...
ZAG0041,2
ZAG0042,1
ZEP0007,5


Which is the total quantity acquired for each `sku`? And the median? And the mean?

In [93]:
df.groupby(['sku']).agg({'product_quantity':['count', 'sum','median','mean']})

product_quantity                     
                     count sum median      mean
sku                                            
8MO0001-A                2   2    1.0  1.000000
8MO0003-A                3   3    1.0  1.000000
8MO0007                 29  30    1.0  1.034483
8MO0008                 30  31    1.0  1.033333
8MO0009                 28  30    1.0  1.071429
...                    ...  ..    ...       ...
ZAG0041                  2   2    1.0  1.000000
ZAG0042                  1   1    1.0  1.000000
ZEP0007                  5   5    1.0  1.000000
ZEP0008                  1   1    1.0  1.000000
par0072                 12  13    1.0  1.083333

[7951 rows x 4 columns]

`groupby` can also be combined with other pandas functions to analyse in more depth the datasets.

In [94]:
df.groupby('sku')[['sku','product_quantity']].describe()

product_quantity                                             
                     count      mean       std  min  25%  50%  75%  max
sku                                                                    
8MO0001-A              2.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
8MO0003-A              3.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
8MO0007               29.0  1.034483  0.185695  1.0  1.0  1.0  1.0  2.0
8MO0008               30.0  1.033333  0.182574  1.0  1.0  1.0  1.0  2.0
8MO0009               28.0  1.071429  0.262265  1.0  1.0  1.0  1.0  2.0
...                    ...       ...       ...  ...  ...  ...  ...  ...
ZAG0041                2.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
ZAG0042                1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0
ZEP0007                5.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
ZEP0008                1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0
par0072               12.0  1.083333  0.288675  1.0  1.0  1.0  1.0  2.0

[7951 rows x 8 columns]

Now let's see how can I combine `.groupby()` and `.aggragate()` with `.sort_values()`.

I would like to see the top 10 most sold products on our data (total product quantity). 

In [95]:
(
df
    .groupby('sku')['product_quantity']
    .sum()
    .sort_values(ascending=False).head(10)
)

sku
APP1190    6366
MIC0036    6316
APP1216    5648
APP0662    5487
APP0663    4164
MMW0016    2615
APP0698    2348
SAT0054    2322
APP1214    1985
WDT0183    1978
Name: product_quantity, dtype: int64

How can I sort values with multiple aggregated functions?

In [96]:
(
df
    .groupby('sku')
    .agg({'product_quantity':['sum','count','std','mean','median']})
    .sort_values(('product_quantity','mean'), ascending=False)
)

product_quantity                                    
                       sum count         std       mean median
sku                                                           
APP1917                 32     1         NaN  32.000000   32.0
ADN0039                132     7   47.245559  18.857143    1.0
KIN0137                862    55  107.718263  15.672727    1.0
EVU0013                177    12   47.005077  14.750000    1.0
SEV0028               1122   122   90.353268   9.196721    1.0
...                    ...   ...         ...        ...    ...
APP1546                  1     1         NaN   1.000000    1.0
APP1546-A                4     4    0.000000   1.000000    1.0
MAK0009                  2     2    0.000000   1.000000    1.0
MAK0008                  1     1         NaN   1.000000    1.0
8MO0001-A                2     2    0.000000   1.000000    1.0

[7951 rows x 5 columns]

## Working with date time

Time to create the week day column. You will have to combine two functions, [`pandas.DataFrame.assign()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html) from `pandas` and [`datetime.strftime()`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) from `datetime`.

First, let's talk about `.assign()`. It is a useful method for creating new columns:

In [97]:
df.assign(new_colum = 'hi! I am a new column!').head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,new_colum
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,hi! I am a new column!
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,hi! I am a new column!
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,hi! I am a new column!
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,hi! I am a new column!
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,hi! I am a new column!


A new column often is the result of an operation between other columns in the dataframe:

In [99]:
# df.assign(total_price = df['product_quantity'] * df['unit_price']).sort_values(by= 'total_price',ascending = False).head()
df.assign(total_price = df['product_quantity'] * df['unit_price']).head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74


In Pandas, we can extract strings containing elements like the day of the month, the day of the week, the hour of the day... from `datetime` objects. We do so by using `dt.strftime()` in combination with `strftime` syntax. Find the cheat sheet for this syntax [here](https://strftime.org/). 

You'll understand it better with an example. Let's create the weekday column:

In [104]:
df.assign(week_day = df['date'].dt.strftime('%A')).head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,week_day
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,Sunday
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,Sunday
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,Sunday
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,Sunday
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,Sunday


Now we can find out which are the week day with more sells in quantity: 

In [64]:
(
df
    .assign(week_day = df['date'].dt.strftime('%A'))
    .groupby('week_day')
    .agg({'product_quantity':['sum','count']})
    .sort_values(('product_quantity','sum'), ascending=False)
)

product_quantity       
                       sum  count
week_day                         
Monday               57624  50307
Wednesday            54131  47550
Tuesday              50871  44498
Friday               49566  44027
Thursday             48431  43634
Sunday               35667  32857
Saturday             33302  31110

Is there a way to find out the top 1 product (in product quantity) by each sku in each weekday?

In [65]:
(
df
    .assign(week_day = df['date'].dt.strftime('%A'))
    .groupby(['week_day','sku'])
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
    .reset_index()
    .groupby('week_day')
    .head(1)
)

,week_day,sku,product_quantity
0,Friday,APP1190,1827
1,Thursday,MIC0036,1389
3,Tuesday,MIC0036,1213
6,Wednesday,SEV0028,1014
10,Monday,APP0663,840
14,Saturday,MIC0036,806
23,Sunday,APP1190,692


# CHALLENGES

1. Which are the top 6 orders with a higher number of products sold?

In [66]:
df.groupby('id_order').agg({'product_quantity':'sum'}).sort_values(('product_quantity'), ascending=False).head(6)

,product_quantity
id_order,
358747,1081
346221,999
349475,800
349133,555
484334,264
395611,256


In [108]:
df.loc[df.id_order == 395611]

,id,id_order,product_id,product_quantity,sku,unit_price,date
111505,1330972,395611,0,1,APP1658,600.33,2017-09-01 20:56:34
111506,1330973,395611,0,1,WOE0013,9.99,2017-09-01 20:56:45
111507,1330974,395611,0,1,JBL0135,129.99,2017-09-01 20:57:07
111508,1330975,395611,0,1,POL0016,26.99,2017-09-01 20:57:19
111509,1330976,395611,0,1,PAC1434,1.555.37,2017-09-01 20:57:25
...,...,...,...,...,...,...,...
111766,1331681,395611,0,1,APP0667,22.00,2017-09-01 21:42:00
111767,1331685,395611,0,1,AP20279,45.00,2017-09-01 21:42:15
111768,1331722,395611,0,1,NTE0068,11.99,2017-09-01 21:45:37
111786,1333589,395611,0,1,BEL0072,2.99,2017-09-01 23:32:36


2. Which are the top 6 orders for the 1 of July of 2017? 

Combining `.assign()` with `.dt.date` will help you extract the date of a datetime column and use this information to filter it. Do some googleing to find out how to use `.dt.date`.

Here is an example how to filter the first of January 2017.

In [67]:
# Example
df_date = df.assign(date = df['date'].dt.date).copy()
df_date[df_date['date'] == pd.to_datetime('2017-01-01')]

C:\Users\HANA\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01
...,...,...,...,...,...,...,...
612,1120279,300119,0,1,JBL0107,94.99,2017-01-01
613,1120287,300126,0,1,LIF0112,61.74,2017-01-01
614,1120289,300127,0,1,EIZ0017,739.99,2017-01-01
615,1120290,300128,0,1,APP1195,1.193.99,2017-01-01


In [68]:
df_date.date.dtype

dtype('O')

### Unit price has to be converted to float

First, I changed format of strings from '1.222.33' to '1222.33'

In [24]:
# first solution(if we assume that we can have prices with 3 or more dots)
#split code for easier explanation
# df['unit_price'] = df['unit_price'].str.rsplit('.',1).apply(lambda x : x[0]+ ',' + x[1])
# df['unit_price'] = df['unit_price'].str.replace('.','',regex=True)
# df['unit_price'] = df['unit_price'].str.replace(',','.',regex=True)

#entire first solution in one line of code
df['unit_price'] = df['unit_price'].str.rsplit('.',1).apply(lambda x : x[0]+ ',' + x[1]).str.replace('.','',regex=True).str.replace(',','.',regex=True)

# another solution (if we assume that there are no prices with 3 or more dots)
# for unit in df['unit_price']:
#     if(unit.count('.') > 1):
#         unit = unit.split('.',1).apply(lambda x : x[0]+ '' + x[1])

In [25]:
print(df['unit_price'].unique())

['18.99' '399.00' '474.05' ... '132.51' '3475.00' '48.68']


Then, I cast it to float

In [71]:
df['unit_price'] = df['unit_price'].astype(float)

In [72]:
df_total_price = df_date[df_date['date'] == pd.to_datetime('2017-07-01')].assign(total_price = df['product_quantity'] * df['unit_price']).copy()
df_total_price.sort_values(by='total_price',ascending=False).head(6)

C:\Users\HANA\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price
83441,1279794,371322,0,3,PAC2107,3.754.00,2017-07-01,11262.00
83236,1279447,371160,0,2,PAC2115,3.499.00,2017-07-01,6998.00
83254,1279475,371178,0,1,APP2352,3.309.00,2017-07-01,3309.00
83151,1279307,371084,0,1,APP2326,3.229.00,2017-07-01,3229.00
83479,1279856,371355,0,8,SEA0098,378.99,2017-07-01,3031.92
83252,1279473,371178,0,1,APP2332,3.001.00,2017-07-01,3001.00


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 15.7+ MB


3. Which is the month with the highest number of units sold? Remember to look at the documentation of [`datetime.strftime()`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) 

In [74]:
(df
    .assign(month = df['date'].dt.strftime('%B'))
    .groupby('month')
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
).head(1)

,product_quantity
month,
January,66415


The results enormous differences between some months. Why do you think this is happening? Do we only have one year of data? In case you have multiple years, filter for only 2017 to find out the best month for that year.

ATTENTION: the output of the function `.dt.strftime()` is a string!

In [75]:
df.date.dt.strftime('%Y').unique()

array(['2017', '2018'], dtype=object)

In [76]:
(df
    .query("date.dt.strftime('%Y') == '2017'")
    .assign( month = df['date'].dt.strftime('%B')) 
    .groupby('month')
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
).head(1)

,product_quantity
month,
November,46375


4. Find the day of the week with the highest amount of products sold by each MONTH. See an idea of the expected output. Take into account the year problem we mentioned before.

In [77]:
pd.DataFrame({
    'month':['November','January','July','...'],
    'week_day':['Friday','Tuesday','Monday','...'],
    'product_quantity':[10000, 9800, 9700, 0]
})

,month,week_day,product_quantity
0,November,Friday,10000
1,January,Tuesday,9800
2,July,Monday,9700
3,...,...,0


In [81]:
(
df
    .assign(week_day = df['date'].dt.strftime('%A'), month = df['date'].dt.strftime('%B'), year = df['date'].dt.strftime('%Y') )
    .groupby(['year','month','week_day'])
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
    .head(10)
)

product_quantity
year month    week_day                   
2017 November Friday                 9717
2018 January  Tuesday                8403
              Monday                 8327
2017 November Thursday               7727
              Wednesday              7137
              Monday                 7109
     December Wednesday              7010
2018 January  Thursday               6952
              Wednesday              6844
2017 December Thursday               5932

In [102]:
(
df
    .assign(week_day = df['date'].dt.strftime('%A'), month = df['date'].dt.strftime('%B'), year = df['date'].dt.strftime('%Y'))
    .groupby(['month', 'week_day', 'year'])
    .agg({'product_quantity':'sum'})
    .sort_values('product_quantity', ascending=False)
    .reset_index()
    .groupby('month')
    .head(1)
)

,month,week_day,year,product_quantity
0,November,Friday,2017,9717
1,January,Tuesday,2018,8403
6,December,Wednesday,2017,7010
20,February,Wednesday,2018,4426
22,July,Monday,2017,4018
23,October,Monday,2017,3849
25,May,Wednesday,2017,3574
32,April,Tuesday,2017,3286
40,March,Monday,2018,3005
43,September,Friday,2017,2893
